# Importing Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install tensorflow_decision_forests

In [ ]:
import tensorflow_decision_forests as tfdf
print("TensorFlow Decision Forests v" + tfdf.__version__)

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
train.head()

# EDA

In [ ]:
print(test.isna().sum())
print(train.isna().sum())

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
#dropping id column

train.drop(['id'],axis=1,inplace=True)
test.drop(['id'],axis=1,inplace=True)


In [ ]:
train.head()

In [ ]:
train['target'].value_counts()

In [ ]:
train.corr()

In [ ]:
train.info()

In [ ]:
float_cols = [f for f in train.columns if train[f].dtype == 'float64']

plt.subplots(figsize=(30,35))
for i,column in enumerate(float_cols):
    plt.subplot(4,4,i+1)
    sns.histplot(data=train,x=column, hue='target')
    plt.title(column)

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(train[float_cols+['target']].corr(),center=0,annot=True,fmt='.2f')
plt.show()

Correlation matrix insights:
1. f_00 to f_06 are connected to f_28
2. f_19 to f_26 are slightly correlated with each other

In [ ]:
int_cols = [f for f in train.columns if train[f].dtype == 'int64']

plt.subplots(figsize=(30,35))
for i,f in enumerate(int_cols):
    val_count = train[f].value_counts()
    ax = plt.subplot(5,3,i+1)
    ax.bar(val_count.index,val_count.values)
    plt.title(f)

Looking at histograms of integer features, first 12 features all have values between 0 and 16. The last 2 features, f_29 is binary and f_30 is ternary.
> Ignore target.

# f_27 - String Feature

f_27 is a string feature which cannot be used as a feature as it is. Let's find a work around.

In [ ]:
train['f_27'].str.len().min(), train.f_27.str.len().max(), test.f_27.str.len().min(), test.f_27.str.len().max()

string has a fixed length of 10 characters.

In [ ]:
train.f_27.value_counts()

In [ ]:
import string
alpha_upper = list(string.ascii_uppercase) #assigning all alphabets as upper case to this list

char_counts=[]
for characters in alpha_upper: #running a for loop through each alphabet
    char_counts.append(train['f_27'].str.count(characters).sum())  #creating a list of sum of all characters present in the column 'f_27'
    
char_counts_df = pd.DataFrame({'Character':alpha_upper, 'Character Count':char_counts}) #Creating a df for the above list
char_counts_df = char_counts_df.loc[char_counts_df['Character Count']>0] #removing the non-existing alphabest from 'f_27'
print(np.sum(char_counts))

plt.subplots(figsize=(20,10))
sns.barplot(data=char_counts_df,y='Character Count',x='Character')
plt.title("Total Number of characters in train[f_27]");

Finding the count of unique characters in string.

In [ ]:
char_counts_df =  char_counts_df.set_index("Character",drop=False)
for i in range(10):
    char_counts_df['character'+str(i+1)] = train['f_27'].str[i].value_counts()
char_counts_df = char_counts_df.fillna(0)

plt.subplots(figsize=(30,12))
characters_cols = [i for i in char_counts_df.columns if 'character' in i] 
for i,f in enumerate(characters_cols):
    plt.subplot(2,5,i+1)
    sns.barplot(data=char_counts_df,x='Character',y=f,color='blue')
    plt.title('Charaacter value counts in position: '+str(i+1));
    

In [ ]:
train['f_00 + f_01'] = train['f_00'] + train['f_01']

In [ ]:
plt.subplots(figsize=(30,30))

plt.subplot(3,3,1)
sns.scatterplot(data=train,y='f_00 + f_01',x='f_26',hue='target',s=2);

plt.subplot(3,3,2)
sns.scatterplot(data=train,y='f_02',x='f_21',hue='target',s=2);

plt.subplot(3,3,3)
sns.scatterplot(data=train,y='f_05',x='f_22',hue='target',s=2);

We need to create a feature which returns a value of 1 for target value 1, 0 for target value 0 & 1, -1 for target value 0

ref - [https://www.kaggle.com/competitions/tabular-playground-series-may-2022/discussion/323892]()

# Feature Engineering

creating a function to feature engineer both train and test data

In [ ]:
train = train.drop(columns=['f_00 + f_01'])
train.info()

In [ ]:
train.shape,test.shape

In [ ]:
def fe(df):
    new_df = df.copy()
    
    #for the 3 features visualized 2 cells above
    
    new_df['i_02_21'] = (df.f_21 + df.f_02 > 5.2).astype(int) - (df.f_21 + df.f_02 < -5.3).astype(int)
    
    new_df['i_05_22'] = (df.f_22 + df.f_05 > 5.2).astype(int) - (df.f_22 + df.f_05 < -5.3).astype(int)
    
    i_00_01_26 = df.f_00 + df.f_01 + df.f_26
    new_df['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.3).astype(int)
    
    #Encoding characters ordinally
    
    for i in range(10):
        new_df['f_27_'+str(i)] = new_df['f_27'].str[i].apply(lambda x:ord(x) - ord('A'))
        
    #getting number of unique characters in 'f_27'
    
    new_df['unique_charaacters'] = new_df['f_27'].apply(lambda x: len(set(x)))
    
    new_df = new_df.drop(columns=['f_27'])
    return new_df

In [ ]:
%%time

train = fe(train)
test = fe(test)

In [ ]:
train.shape,test.shape

# RandomForest Model


In [ ]:
#loading data into a TF dataset

train_tfds = tfdf.keras.pd_dataframe_to_tf_dataset(train,label='target')
test_tfds = tfdf.keras.pd_dataframe_to_tf_dataset(test)

In [ ]:
print(tfdf.keras.RandomForestModel.predefined_hyperparameters())

In [ ]:
#Train the model

rf_model = tfdf.keras.RandomForestModel(hyperparameter_template="better_default")
rf_model.compile(metrics=['AUC']) 
rf_model.fit(x=train_tfds)

In [ ]:
tfdf.model_plotter.plot_model_in_colab(rf_model, tree_idx=0, max_depth=3)

In [ ]:
inspector = rf_model.make_inspector()
inspector.evaluation()

In [ ]:
rf_model.evaluate(train_tfds)

In [ ]:
print("Model type:", inspector.model_type())
print("Objective:", inspector.objective())
print("Evaluation:", inspector.evaluation())

In [ ]:
sample_sub = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv')
sample_sub['target'] = rf_model.predict(test_tfds)
sample_sub.to_csv('submission.csv',index=False)
sample_sub.head()